**Capstone Project Group A CV1**

## Pneumonia Detection Challenge
The goal is to build a pneumonia detection system, to locate the position of inflammation in an image.

This model is built using Detectron2 (Faster_RCNN) on pytorch: https://github.com/facebookresearch/detectron2


**Data preprocessing steps needed are as below:**
1. Convert all the dicom images to jpg as Detectron2 can't directly read dicom images  : *convert_to_jpg()*
2. Convert the data to coco json format : *get_data_dicts()*

In [ ]:
!pip install pyyaml==5.1
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
!pip install pydicom
!pip install sklearn
!pip install funcy
!pip install argparse

In [44]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

1.9.0+cu111 True


In [45]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import glob, os, json, cv2, random
from google.colab.patches import cv2_imshow
import pydicom as pyd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [46]:
base_dir='/content/drive/MyDrive/colab'
data_dir = base_dir+'/input'
working_dir = base_dir+'/working'

train_labels_df = pd.read_csv(data_dir+'/stage_2_train_labels.csv')
label_meta_data = pd.read_csv(data_dir+'/stage_2_detailed_class_info.csv')

train_dicom_dir = os.path.join(data_dir, 'stage_2_train_images')
test_dicom_dir = os.path.join(data_dir, 'stage_2_test_images')

train_labels_df.tail(10)

,patientId,x,y,width,height,Target
30217,c1e228e4-b7b4-432b-a735-36c48fdb806f,NaN,NaN,NaN,NaN,0
30218,c1e3eb82-c55a-471f-a57f-fe1a823469da,NaN,NaN,NaN,NaN,0
30219,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,666.0,418.0,186.0,223.0,1
30220,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,316.0,504.0,179.0,273.0,1
30221,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,609.0,464.0,240.0,284.0,1
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1
30226,c1f7889a-9ea9-4acb-b64c-b737c929599a,233.0,424.0,201.0,356.0,1


In [47]:
def get_files(dicom_dir):
    files = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(files))

def parse_dataset(dicom_dir, anns): 
    image_files = get_files(dicom_dir)
    image_annotations = {fp: [] for fp in image_files}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
        if index == 0:
          print(row)
    return image_files, image_annotations 

image_files, image_annotations = parse_dataset(train_dicom_dir, anns=train_labels_df)

patientId    0004cfab-14fd-4e49-80ba-63a80b6bddd6
x                                             NaN
y                                             NaN
width                                         NaN
height                                        NaN
Target                                          0
Name: 0, dtype: object


In [48]:
from detectron2.structures import BoxMode

def get_data_dicts(img_dir):
    dataset_dicts = []
    for idx, arr in enumerate(image_annotations.values()):
        record = {}
        filename = os.path.join(img_dir, arr[0]["patientId"]+".jpg")
        img = cv2.imread(filename)
        if img is None:
          continue;
        height, width = img.shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        objs = []
        for v in arr: 
          if np.isnan(float(v["x"])):
            continue
          obj = {
                  "bbox": [v["x"], v["y"], v["width"], v["height"]],
                  "bbox_mode": BoxMode.XYWH_ABS,
                  "category_id": 0,
              }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
          
        # if idx > 5000:
        #   break;
    return dataset_dicts

### Create dataset and save

In [49]:
import json
out_file = open(os.path.join(data_dir,"coco-full.json"), "w")
json.dump(get_data_dicts(data_dir+'/train'), out_file, indent = 4) 
out_file.close()

### Split the dataset into train and validation

In [50]:
with open(os.path.join(data_dir,"coco-full.json"), 'r') as myfile:
    data=myfile.read()

# parse file
obj = json.loads(data)
 
X_train, X_val = train_test_split(obj, test_size=0.2)
print(len(X_train))
print(len(X_val))

21347
5337


In [ ]:
def get_dicts(name):
    return X_train if name == 'train' else X_val

for d in ["train", "val"]:
    try:
      DatasetCatalog.register(d, lambda d=d: get_dicts(d))
      MetadataCatalog.get(d).set(thing_classes=["pneumonia"])
    except:
      print('')

In [55]:
pnuemonia_metadata = MetadataCatalog.get("train")
print( MetadataCatalog.get("train"))

Metadata(name='train', thing_classes=['pneumonia'])


In [56]:
def convert_to_jpg(input_file, output_file):
    ds = pyd.dcmread(input_file)
    cv2.imwrite(output_file, ds.pixel_array)

# for d in ["test"]:
#     input_path = os.path.join(data_dir, "stage_2_"+d+"_images")
   
#     for file in get_files(input_path):
#          output_file = os.path.join(data_dir, d, os.path.basename(file).replace(".dcm", "")+".jpg")
#          convert_to_jpg(file, output_file)

In [57]:
def show(d):  
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=pnuemonia_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [58]:
for d in random.sample(X_train, 15):
    show(d)

Output hidden; open in https://colab.research.google.com to view.

In [59]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"))
cfg.DATASETS.TRAIN = ("train")
cfg.DATASETS.TEST = ()
cfg.OUTPUT_DIR = ( base_dir + "/output")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 500    
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[10/30 11:25:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[10/30 11:25:42 d2.engine.train_loop]: Starting training from iteration 0
[10/30 11:26:00 d2.utils.events]:  eta: 0:07:09  iter: 19  total_loss: 1.119  loss_cls: 0.6855  loss_box_reg: 0.08186  loss_rpn_cls: 0.3146  loss_rpn_loc: 0.03214  time: 0.8944  data_time: 0.0123  lr: 9.7405e-06  max_mem: 14568M
[10/30 11:26:18 d2.utils.events]:  eta: 0:06:51  iter: 39  total_loss: 0.8908  loss_cls: 0.4771  loss_box_reg: 0.09476  loss_rpn_cls: 0.2819  loss_rpn_loc: 0.02906  time: 0.8939  data_time: 0.0043  lr: 1.9731e-05  max_mem: 14568M
[10/30 11:26:36 d2.utils.events]:  eta: 0:06:33  iter: 59  total_loss: 0.6202  loss_cls: 0.2777  loss_box_reg: 0.07999  loss_rpn_cls: 0.2249  loss_rpn_loc: 0.02663  time: 0.8957  data_time: 0.0045  lr: 2.972e-05  max_mem: 14568M
[10/30 11:26:54 d2.utils.events]:  eta: 0:06:15  iter: 79  total_loss: 0.569  loss_cls: 0.1767  loss_box_reg: 0.09895  loss_rpn_cls: 0.2144  loss_rpn_loc: 0.03299  time: 0.8948  data_time: 0.0042  lr: 3.9711e-05  max_mem: 14568M
[10/30 11

## Inference & evaluation using the trained model

In [60]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [61]:
result = [];
for d in X_val:      
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    no_objects = len(outputs["instances"].pred_boxes)

    predicted = 1 if no_objects > 0 else 0
    ground_truth = 1 if len(d["annotations"]) > 0 else 0
    result.append({"id":  os.path.basename(d["file_name"]).replace(".jpg", ""), "ground_truth": ground_truth, "predicted": predicted})



In [62]:
df = pd.DataFrame(result)
df.head()

,id,ground_truth,predicted
0,ce894c80-15b8-4e95-98a2-49f09bddd6b1,0,0
1,fc0e9c65-29d3-4950-9565-b8f44f7f5dca,0,0
2,fbf900a9-0991-45d9-8bdc-b451f2671492,0,0
3,3ea80a73-44fc-4b9e-946a-ebd620e004fe,0,0
4,b3a287c5-9fc6-42fb-93c5-310515024948,1,0
...,...,...,...
95,f7b3faca-1b1d-4428-9dc2-086106c871bf,0,0
96,1953e32b-c76f-4ff4-a893-484772972100,0,0
97,c7e641a6-a80b-4314-961d-d63e4a54d5bc,1,0
98,4efadc3e-4c7d-4bfc-b46e-fe3b1cd31488,0,0


In [68]:
print(classification_report(df['ground_truth'], df['predicted']))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88      4143
           1       0.60      0.35      0.44      1194

    accuracy                           0.80      5337
   macro avg       0.72      0.64      0.66      5337
weighted avg       0.78      0.80      0.78      5337



In [69]:
accuracy = accuracy_score(df['ground_truth'], df['predicted'])
print('Model accuracy is: ', accuracy)

Model accuracy is:  0.8019486602960465


### Make Predictions

In [65]:
def get_test_files(dir):
    files = glob.glob(dir+'/'+'*.jpg')
    return list(set(files))


In [73]:
from detectron2.utils.visualizer import ColorMode

for file in random.sample(get_test_files(data_dir+'/test'), 10):      
    im = cv2.imread(file)
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=pnuemonia_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))   
    cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.